In [1]:
# test pipeline for the bruyere dataset
from fn_cfg import *
import params as cfg

In [ ]:
localPath = '/Volumes/Backup Plus/EEG_Datasets/laurel_place/dataset'
filename = '0002_2_12122019_1225'
version = 1.0

#version = 1.1
#filename = '0_1_12072018_1206'
#localPath = '/Users/joshuaighalo/Downloads/raw'

device = importFile.neurocatch()
fileObjects = device.init(version,filename,localPath)
rawEEG = fileObjects[0]
rawEOG = fileObjects[1]
rawEEGEOG = fileObjects[2]
time = fileObjects[3]
trigOutput = fileObjects[4]
plots(time,rawEEG,titles=cfg.channelNames,figsize=cfg.figure_size,pltclr=cfg.plot_color)
spectogramPlot(rawEEG,fs,nfft=cfg.nfft,nOverlap=cfg.noverlap,figsize=(16,6),subTitles=cfg.channelNames,title='Music Therapy Group 11')
filtering = filters()
notchFilterOutput = filtering.notch(rawEEG,line,fs)
plots(time,notchFilterOutput,titles=cfg.channelNames,figsize=cfg.figure_size,pltclr=cfg.plot_color)
#spectogramPlot(notchFilterOutput,fs,nfft=cfg.nfft,nOverlap=cfg.noverlap,figsize=(16,6),subTitles=cfg.channelNames,title='Music Therapy Group 11')


In [ ]:
" Functions Utilized"

#   Comparative Study of Wavelet-Based Unsupervised Ocular Artifact Removal Techniques for Single-Channel EEG Data
#   Signal to Artifact Ratio (SAR) is a quantification method to measure the amount of artifact removal 
#   in a specific signal after processing with an algorithm [40].
#   SAR is a measure of the amount of artifact removal in a signal.
#   x = EEG signal containing artifact
#   y = EEG signal obtained after running an artifact free algorithm
def sar(x,y):
    return 10*np.log10((np.std(x))/(np.std(x-y)))
def mse(x,y):
    return math.sqrt(np.square(np.subtract(x,y)).mean())

def DWT(data,time_array):
    #   Probability Mapping Based Artifact Detection and Wavelet Denoising based 
    #   Artifact Removal from Scalp EEG for BCI Applications
    #  Perform DWT on the data
    #   Input: data - EEG data: 1D array (samples x channel)
    #   Output: new signal: (samples x number of wavelets)
    #           signal_global - new signal extracted after global threshold 
    #           signal_std - new signal extracted after std threshold 
    #   Reference:  choice of number of levels to threshold gotten from "Comparative Study of Wavelet-Based Unsupervised 
    #               Ocular Artifact Removal Techniques for Single-Channel EEG Data"
      
    wavelets = ['sym3','coif3','haar','bior4.4'] 
    def dwt_only(data,wavelet):
        def dwt(data,wavelet):
            coeffs = wavedec(data,wavelet,level=10)
            return np.array(coeffs,dtype=object).T

        def global_threshold(data,coeffs):
            def coeffs_approx(data,coeffs):
                return (np.median(abs(coeffs[0]))/0.6745)*(np.sqrt(2*np.log(len(data))))
            def coeffs_detail(data,coeffs):
                return (np.median(abs(coeffs[1]))/0.6745)*(np.sqrt(2*np.log(len(data))))
            arr_approx = coeffs_approx(data,coeffs)
            arr_detail = coeffs_detail(data,coeffs)
            return np.vstack((arr_approx,arr_detail))

        def apply_threshold(coeffs,threshold):
            def apply_threshold_approx(coeffs,threshold):
                coeffs[0][abs(coeffs[0])>threshold[0]] = 0
                coeffs_approx = coeffs[0]
                return coeffs_approx
            def apply_threshold_detail(coeffs,threshold):
                coeffs = coeffs[1:len(coeffs)]
                #coeffs[0][abs(coeffs[0])>threshold[1]] = 0
                #coeffs[1][abs(coeffs[1])>threshold[1]] = 0
                coeffs[2][abs(coeffs[2])>threshold[1]] = 0  # level 8
                coeffs[3][abs(coeffs[3])>threshold[1]] = 0  # level 7
                coeffs[4][abs(coeffs[4])>threshold[1]] = 0  # level 6
                coeffs[5][abs(coeffs[5])>threshold[1]] = 0  # level 5
                coeffs[6][abs(coeffs[6])>threshold[1]] = 0  # level 4
                coeffs[7][abs(coeffs[7])>threshold[1]] = 0  # level 3
                #coeffs[8][abs(coeffs[8])>threshold[1]] = 0
                #coeffs[9][abs(coeffs[9])>threshold[1]] = 0
                return coeffs
            arr_approx = apply_threshold_approx(coeffs,threshold)
            arr_detail = apply_threshold_detail(coeffs,threshold)
            arr_detail = list(np.array(arr_detail).T)
            arr_approx = arr_approx
            coefs = arr_detail
            (coefs).insert(0,arr_approx)
            return coefs

        def inv_dwt(coeffs,wavelet):
            def inverse_dwt(coeffs,wavelet):
                return waverec(coeffs,wavelet)
            arr = (inverse_dwt(list(np.array(coeffs,dtype=object)),wavelet))
            return  (np.array(arr).T)[:-1]

        coeffs = dwt(data,wavelet)
        threshold_global = global_threshold(data,coeffs)
        coeffs_global = apply_threshold(coeffs,threshold_global)
        signal_global = inv_dwt(coeffs_global,wavelet)
        return signal_global

    newEEG_global = []
    for i in range(len(wavelets)):
        newEEG_global.append((dwt_only(data,wavelets[i])))
    newEEG_global = np.array(newEEG_global).T
    if len(newEEG_global) != len(time_array):
        if len(newEEG_global) > len(time_array):
            diff = len(newEEG_global) - len(time_array)
            newEEG_global = newEEG_global[:-diff,:]
        elif len(newEEG_global) < len(time_array):
            diff = len(time_array) - len(newEEG_global)
            num_zeros = np.zeros((diff,len(newEEG_global[1])))
            newEEG_global = np.append(newEEG_global,num_zeros,axis=0)
    else:
        newEEG_global = newEEG_global
    return newEEG_global

In [ ]:
"DWT only"
data = notchFilterOutput[:,1]
dwt_eeg_gt = DWT(data,time)
plots(time,dwt_eeg_gt,['sym3','coif3','haar','bior4.4'],(20,10),['r','g','b','y'])

print("SAR sym3 for global threshold: ", round(sar(data,dwt_eeg_gt[:,0]),3))
print("SAR coif3 for global threshold: ", round(sar(data,dwt_eeg_gt[:,1]),3))
print("SAR haar for global threshold: ", round(sar(data,dwt_eeg_gt[:,2]),3))
print("SAR bior4.4 for global threshold: ", round(sar(data,dwt_eeg_gt[:,3]),3))
print("MSE sym3 for global threshold: ", round(mse(data,dwt_eeg_gt[:,0]),3))
print("MSE coif3 for global threshold: ", round(mse(data,dwt_eeg_gt[:,1]),3))
print("MSE haar for global threshold: ", round(mse(data,dwt_eeg_gt[:,2]),3))
print("MSE bior4.4 for global threshold: ", round(mse(data,dwt_eeg_gt[:,3]),3))

spectogramPlot(dwt_eeg_gt,fs,nfft=cfg.nfft,nOverlap=cfg.noverlap,figsize=(16,6),subTitles=['sym3','coif3','haar','bior4.4'],title='Output')

In [ ]:
"DWT + BandPass Filter"

bandPassFilterOutput = filtering.butterBandPass(dwt_eeg_gt,lowcut=cfg.highPass,highcut=cfg.lowPass,fs=cfg.fs)
plots(time,bandPassFilterOutput,titles=['sym3','coif3','haar','bior4.4'],figsize=(16,6),pltclr=['r','g','b','y'])

print("SAR sym3 for global threshold: ", round(sar(data,bandPassFilterOutput[:,0]),3))
print("SAR coif3 for global threshold: ", round(sar(data,bandPassFilterOutput[:,1]),3))
print("SAR haar for global threshold: ", round(sar(data,bandPassFilterOutput[:,2]),3))
print("SAR bior4.4 for global threshold: ", round(sar(data,bandPassFilterOutput[:,3]),3))
print("MSE sym3 for global threshold: ", round(mse(data,bandPassFilterOutput[:,0]),3))
print("MSE coif3 for global threshold: ", round(mse(data,bandPassFilterOutput[:,1]),3))
print("MSE haar for global threshold: ", round(mse(data,bandPassFilterOutput[:,2]),3))
print("MSE bior4.4 for global threshold: ", round(mse(data,bandPassFilterOutput[:,3]),3))

spectogramPlot(bandPassFilterOutput,fs,nfft=cfg.nfft,nOverlap=cfg.noverlap,figsize=(16,6),subTitles=['sym3','coif3','haar','bior4.4'],title='Output')

print("\n")

data_ = bandPassFilterOutput[:,2]
data_ = data_.reshape(data_.shape[0],1)
erps = erpExtraction()
N1P3 = erps.N100P300(trigOutput,data_,time,stimTrig=cfg.stimTrig,clip=cfg.clip)
N4 = erps.N400(trigOutput,data_,time,stimTrig=cfg.stimTrig,clip=cfg.clip)
N1P3_chan = N1P3[0]
N4_chan = N4[0]
erp_latency = np.array(np.linspace(start=-100, stop=900, num=len(N1P3_chan[0]),dtype=object),dtype=object)
plot_ERPs(N1P3_chan[0],N1P3_chan[1],erp_latency,'N1P3_chan','Latency (ms)','Amplitude (uV)','std','dev','b','r',10)
plot_ERPs(N4_chan[0],N4_chan[1],erp_latency,'N4_chan','Latency (ms)','Amplitude (uV)','std','dev','b','r',10)

In [ ]:
"DWT + Adaptive filter"

adaptiveFilterOutput = filtering.adaptive(dwt_eeg_gt,rawEOG)
plots(time,adaptiveFilterOutput,titles=['sym3','coif3','haar','bior4.4'],figsize=(16,6),pltclr=['r','g','b','y'])

print("SAR sym3 for global threshold: ", round(sar(data,adaptiveFilterOutput[:,0]),3))
print("SAR coif3 for global threshold: ", round(sar(data,adaptiveFilterOutput[:,1]),3))
print("SAR haar for global threshold: ", round(sar(data,adaptiveFilterOutput[:,2]),3))
print("SAR bior4.4 for global threshold: ", round(sar(data,adaptiveFilterOutput[:,3]),3))
print("MSE sym3 for global threshold: ", round(mse(data,adaptiveFilterOutput[:,0]),3))
print("MSE coif3 for global threshold: ", round(mse(data,adaptiveFilterOutput[:,1]),3))
print("MSE haar for global threshold: ", round(mse(data,adaptiveFilterOutput[:,2]),3))
print("MSE bior4.4 for global threshold: ", round(mse(data,adaptiveFilterOutput[:,3]),3))

spectogramPlot(adaptiveFilterOutput,fs,nfft=cfg.nfft,nOverlap=cfg.noverlap,figsize=(16,6),subTitles=['sym3','coif3','haar','bior4.4'],title='Output')

print("\n")

data_ = adaptiveFilterOutput[:,2]
data_ = data_.reshape(data_.shape[0],1)
erps = erpExtraction()
N1P3 = erps.N100P300(trigOutput,data_,time,stimTrig=cfg.stimTrig,clip=cfg.clip)
N4 = erps.N400(trigOutput,data_,time,stimTrig=cfg.stimTrig,clip=cfg.clip)
N1P3_chan = N1P3[0]
N4_chan = N4[0]
erp_latency = np.array(np.linspace(start=-100, stop=900, num=len(N1P3_chan[0]),dtype=object),dtype=object)
plot_ERPs(N1P3_chan[0],N1P3_chan[1],erp_latency,'N1P3_chan','Latency (ms)','Amplitude (uV)','std','dev','b','r',10)
plot_ERPs(N4_chan[0],N4_chan[1],erp_latency,'N4_chan','Latency (ms)','Amplitude (uV)','std','dev','b','r',10)

In [ ]:
"DWT + BandPass + Adaptive Filter"
adaptiveFilterOutput = filtering.adaptive(bandPassFilterOutput,rawEOG)
plots(time,adaptiveFilterOutput,titles=['sym3','coif3','haar','bior4.4'],figsize=(16,6),pltclr=['r','g','b','y'])

print("SAR sym3 for global threshold: ", round(sar(data,adaptiveFilterOutput[:,0]),3))
print("SAR coif3 for global threshold: ", round(sar(data,adaptiveFilterOutput[:,1]),3))
print("SAR haar for global threshold: ", round(sar(data,adaptiveFilterOutput[:,2]),3))
print("SAR bior4.4 for global threshold: ", round(sar(data,adaptiveFilterOutput[:,3]),3))
print("MSE sym3 for global threshold: ", round(mse(data,adaptiveFilterOutput[:,0]),3))
print("MSE coif3 for global threshold: ", round(mse(data,adaptiveFilterOutput[:,1]),3))
print("MSE haar for global threshold: ", round(mse(data,adaptiveFilterOutput[:,2]),3))
print("MSE bior4.4 for global threshold: ", round(mse(data,adaptiveFilterOutput[:,3]),3))

spectogramPlot(adaptiveFilterOutput,fs,nfft=cfg.nfft,nOverlap=cfg.noverlap,figsize=(16,6),subTitles=['sym3','coif3','haar','bior4.4'],title='Output')

print("\n")

data_ = adaptiveFilterOutput[:,2]
data_ = data_.reshape(data_.shape[0],1)
erps = erpExtraction()
N1P3 = erps.N100P300(trigOutput,data_,time,stimTrig=cfg.stimTrig,clip=cfg.clip)
N4 = erps.N400(trigOutput,data_,time,stimTrig=cfg.stimTrig,clip=cfg.clip)
N1P3_chan = N1P3[0]
N4_chan = N4[0]
erp_latency = np.array(np.linspace(start=-100, stop=900, num=len(N1P3_chan[0]),dtype=object),dtype=object)
plot_ERPs(N1P3_chan[0],N1P3_chan[1],erp_latency,'N1P3_chan','Latency (ms)','Amplitude (uV)','std','dev','b','r',10)
plot_ERPs(N4_chan[0],N4_chan[1],erp_latency,'N4_chan','Latency (ms)','Amplitude (uV)','std','dev','b','r',10)